In [1]:
import os
import wandb
import numpy as np
import torch
import collections
import pathlib
import h5py
import dill
import math
import wandb.sdk.data_types.video as wv
from diffusion_policy.gym_util.async_vector_env import AsyncVectorEnv
# from diffusion_policy.gym_util.sync_vector_env import SyncVectorEnv
from diffusion_policy.gym_util.multistep_wrapper import MultiStepWrapper
from diffusion_policy.gym_util.video_recording_wrapper import VideoRecordingWrapper, VideoRecorder
from diffusion_policy.model.common.rotation_transformer import RotationTransformer

from diffusion_policy.policy.base_lowdim_policy import BaseLowdimPolicy
from diffusion_policy.common.pytorch_util import dict_apply
from diffusion_policy.env_runner.base_lowdim_runner import BaseLowdimRunner
from diffusion_policy.env.robomimic.robomimic_lowdim_wrapper import RobomimicLowdimWrapper
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils


import os
import pathlib
import click
import hydra
import torch
import dill
import wandb
import json
from diffusion_policy.workspace.base_workspace import BaseWorkspace


ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /c2/namsan/samsung_recipe/lib/robomimic/robomimic/scripts/setup_macros.py
)


In [2]:
checkpoint = "/c2/namsan/samsung_recipe/lib/diffusion_policy/weights/low_dim/transport_ph/diffusion_policy_cnn/train_2/checkpoints/epoch=2800-test_mean_score=1.000.ckpt"
data_path = "/c2/namsan/dataset/robomimic/datasets/transport/ph/low_dim.hdf5"
output_dir = 'vis'
device = 'cuda:0'

# if os.path.exists(output_dir):
#     click.confirm(f"Output path {output_dir} already exists! Overwrite?", abort=True)
pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

# load checkpoint
payload = torch.load(open(checkpoint, 'rb'), pickle_module=dill)
cfg = payload['cfg']

#cfg.training.seed = None

cls = hydra.utils.get_class(cfg._target_)
workspace = cls(cfg, output_dir=output_dir)
workspace: BaseWorkspace
workspace.load_payload(payload, exclude_keys=None, include_keys=None)

cfg['task']['dataset_path'] = data_path
cfg['task']['env_runner']['dataset_path'] = data_path
cfg['task']['env_runner']['n_train'] = 0
cfg['task']['env_runner']['n_test'] = 0
cfg['task']['env_runner']['n_envs'] = 20

cfg['task']['env_runner']['n_train_vis'] = 0
cfg['task']['env_runner']['n_test_vis'] = 0

In [3]:
# run eval
env_runner = hydra.utils.instantiate(
    cfg.task.env_runner,
    output_dir=output_dir
)

Created environment with name TwoArmTransport
Action size is 14
ROBOMIMIC WARNING(
    No environment version found in dataset!
    Cannot verify if dataset and installed environment versions match
)
Created environment with name TwoArmTransport
Action size is 14
ROBOMIMIC WARNING(
    No environment version found in dataset!
    Cannot verify if dataset and installed environment versions match
)
Created environment with name TwoArmTransportCreated environment with name TwoArmTransport

Action size is 14
Action size is 14
ROBOMIMIC WARNING(
    No environment version found in dataset!
    Cannot verify if dataset and installed environment versions match
)ROBOMIMIC WARNING(
    No environment version found in dataset!
    Cannot verify if dataset and installed environment versions match
)Created environment with name TwoArmTransport
Action size is 14


ROBOMIMIC WARNING(
    No environment version found in dataset!
    Cannot verify if dataset and installed environment versions match
)C

In [4]:
def make_task(seed, render=False):
    def init_fn(env, seed=seed, 
        enable_render=render):
        # setup rendering
        # video_wrapper
        assert isinstance(env.env, VideoRecordingWrapper)
        env.env.video_recoder.stop()
        env.env.file_path = None
        if enable_render:
            filename = pathlib.Path(output_dir).joinpath(
                'media', wv.util.generate_id() + ".mp4")
            filename.parent.mkdir(parents=False, exist_ok=True)
            filename = str(filename)
            env.env.file_path = filename

        # switch to seed reset
        assert isinstance(env.env.env, RobomimicLowdimWrapper)
        env.env.env.init_state = None
        env.seed(seed)
    return dill.dumps(init_fn)

In [5]:
test_tasks = [
    make_task(cfg.task.env_runner.test_start_seed + i)
    for i in range(20)
]

In [9]:
# get policy from workspace
policy = workspace.model
if cfg.training.use_ema:
    policy = workspace.ema_model

device = torch.device(device)
policy.to(device)
policy.eval()
None

# Init-policy

In [7]:
from tqdm import tqdm, trange
from scipy import interpolate

from simpl.collector.storage import Episode

def collect_episodes(policy, tasks):
    n_envs = len(env_runner.env_fns)
    if len(tasks) % n_envs == 0:
        n_redundant = 0
    else:
        n_redundant = n_envs - len(tasks) % n_envs
    init_fns = tasks + [tasks[-1]] * n_redundant

    episodes = []
    for i in trange(len(init_fns) // n_envs):
        episodes_chunk = collect_episodes_(policy, init_fns[i*n_envs:(i+1)*n_envs])
        episodes.extend(episodes_chunk)
    return episodes[:len(tasks)]
    

def collect_episodes_(policy, init_fns):
    n_envs = len(env_runner.env_fns)
    assert len(init_fns) == n_envs
    
    device = policy.device
    dtype = policy.dtype
    env = env_runner.env

    # init envs
    env.call_each('run_dill_function', 
        args_list=[(x,) for x in init_fns]
    )
    
    # start rollout
    obs = env.reset()
    past_action = None
    policy.reset()

    full_obs = env_runner.env.call('get_attr', 'obs')
    episodes = [
        Episode(np.stack(full_obs[i])[-env_runner.n_action_steps:, :][0])
        for i in range(n_envs)
    ]
    
    done = False

    while not done:
        # create obs dict
        np_obs_dict = {
            # handle n_latency_steps by discarding the last n_latency_steps
            'obs': obs[:,:env_runner.n_obs_steps].astype(np.float32)
        }
        if env_runner.past_action and (past_action is not None):
            # TODO: not tested
            np_obs_dict['past_action'] = past_action[
                :,-(env_runner.n_obs_steps-1):].astype(np.float32)
        
        # device transfer
        obs_dict = dict_apply(
            np_obs_dict, 
            lambda x: torch.from_numpy(x).to(device=device)
        )
    
        # run policy
        with torch.no_grad():
            action_dict = policy.predict_action(obs_dict)

        # device_transfer
        np_action_dict = dict_apply(action_dict,
            lambda x: x.detach().to('cpu').numpy())
        
        # handle latency_steps, we discard the first n_latency_steps actions
        # to simulate latency
        action = np_action_dict['action'][:,env_runner.n_latency_steps:]
        if not np.all(np.isfinite(action)):
            print(action)
            raise RuntimeError("Nan or Inf action")
        
        # action = action_dict['action_pred'].detach().cpu().numpy()
        
        # assert policy.oa_step_convention
        # action_start_t = policy.n_obs_steps - 1
        # action = action[:, action_start_t:action_start_t+policy.n_action_steps]
        
        # assert env_runner.n_latency_steps == 0
    
        # step env
        env_action = action
        if env_runner.abs_action:
            env_action = env_runner.undo_transform_action(action)
    
        obs, reward, done, info = env.step(env_action)
        
        # full_obs = env_runner.env.call('get_attr', 'obs')
        for i, episode in enumerate(episodes):
            obss = info[i]['single_step_obs']
            rewards = info[i]['single_step_reward']
            l = len(obss)
            dones = [False] * (l-1) + [done[i]]  # infer when exactly it's done

            for t in range(l):
                if len(episode) > 0 and episode.dones[-1] == True:  # do nothing if it's already done
                    break
                    
                dd = (rewards[t] > 0)  # CAUTION : Hard-coded termination on succeed
                
                episode.add_step(action[i, t], obss[t], rewards[t], dd, None)
        
        done = np.all(done | np.array([episode.dones[-1] for episode in episodes]))
        past_action = action
    
    # env.render()
    
    return episodes


In [8]:
N = 10

all_episodes = []
success_rates = []
execution_times = []
for i in range(N):
    episodes = collect_episodes(policy, test_tasks)
    all_episodes.append(episodes)
    success_rate = np.mean([
        epi.rewards[-1] == 1
        for epi in episodes
    ])
    execution_time = np.mean([
        len(epi)
        for epi in episodes
        if epi.rewards[-1] == 1
    ]) / 20
    
    success_rates.append(success_rate)
    execution_times.append(execution_time)
    print(i, success_rate, execution_time)
print('success_rate', np.mean(success_rates), np.std(success_rates) / np.sqrt(N))
print('execution_time', np.mean(execution_times), np.std(execution_times) / np.sqrt(N))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [04:04<00:00, 244.31s/it]


0 1.0 22.315


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [04:58<00:00, 298.57s/it]


1 0.9 22.197222222222223


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [04:03<00:00, 243.84s/it]


...

3 0.85 22.761764705882353


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [04:10<00:00, 250.40s/it]


4 0.95 23.076315789473686


  0%|                                                                                                                  | 0/1 [01:35<?, ?it/s]


KeyboardInterrupt: 

# FF Policy

In [10]:
from tqdm import tqdm, trange
from scipy import interpolate

from simpl.collector.storage import Episode


SPEED_FACTOR = 1.25

def collect_episodes_ff(policy, ff_policy, tasks):
    n_envs = len(env_runner.env_fns)
    if len(tasks) % n_envs == 0:
        n_redundant = 0
    else:
        n_redundant = n_envs - len(tasks) % n_envs
    init_fns = tasks + [tasks[-1]] * n_redundant

    episodes, ff_episodes = [], []
    for i in trange(len(init_fns) // n_envs):
        episodes_chunk, ff_episodes_chunk = collect_episodes_ff_(policy, ff_policy, init_fns[i*n_envs:(i+1)*n_envs])
        episodes.extend(episodes_chunk)
        ff_episodes.extend(ff_episodes_chunk)
    return episodes[:len(tasks)], ff_episodes[:len(tasks)]


def collect_episodes_ff_(policy, ff_policy, init_fns):
    n_envs = len(env_runner.env_fns)
    assert len(init_fns) == n_envs
    
    device = policy.device
    dtype = policy.dtype
    env = env_runner.env

    # init envs
    env.call_each('run_dill_function', 
        args_list=[(x,) for x in init_fns]
    )
    
    # start rollout
    obs = env.reset()
    past_action = None
    policy.reset()

    full_obs = env_runner.env.call('get_attr', 'obs')
    episodes = [
        Episode(np.stack(full_obs[i])[-env_runner.n_action_steps:, :][0])
        for i in range(n_envs)
    ]
    ff_episodes = [
        Episode(np.stack(obs[i]))
        for i in range(n_envs)
    ]
    
    done = False

    while not done:
        # create obs dict
        np_obs_dict = {
            # handle n_latency_steps by discarding the last n_latency_steps
            'obs': obs[:,:env_runner.n_obs_steps].astype(np.float32)
        }
        if env_runner.past_action and (past_action is not None):
            # TODO: not tested
            np_obs_dict['past_action'] = past_action[
                :,-(env_runner.n_obs_steps-1):].astype(np.float32)
        
        # device transfer
        obs_dict = dict_apply(
            np_obs_dict,
            lambda x: torch.from_numpy(x).to(device=device)
        )
    
        # run policy
        with torch.no_grad():
            action_dict = policy.predict_action(obs_dict)
            ff_action = ff_policy.act(
                policy.normalizer['obs'](obs_dict['obs'])
            )
        ffd_action = []
        
        assert policy.oa_step_convention
        action_start_t = policy.n_obs_steps - 1
        action_pred = action_dict['action_pred'].detach().cpu().numpy()
        for a, ff_a in zip(action_pred, ff_action):
            if ff_a == 0:
                ffd_action.append(
                    a[action_start_t:action_start_t+policy.n_action_steps]
                )
            elif ff_a == 1:
                ts = np.arange(policy.horizon)
                f = interpolate.interp1d(ts, a, axis=0)
                
                ff_ts = ts[action_start_t:]
                ff_ts = SPEED_FACTOR*ff_ts[:policy.n_action_steps]
                
                ffd_action.append(
                    f(ff_ts)
                )
                assert (ffd_action[-1].shape[0] == policy.n_action_steps), (a.shape, ffd_action[-1].shape)
            else:
                raise
        ffd_action = np.stack(ffd_action)
    
        # device_transfer
        # np_action_dict = dict_apply(action_dict,
        #     lambda x: x.detach().to('cpu').numpy())
        
        # handle latency_steps, we discard the first n_latency_steps actions
        # to simulate latency
        action = ffd_action
        #action = np_action_dict['action'][:,env_runner.n_latency_steps:]
        # if not np.all(np.isfinite(action)):
        #     print(action)
        #     raise RuntimeError("Nan or Inf action")
    
        # step env
        env_action = action
        if env_runner.abs_action:
            env_action = env_runner.undo_transform_action(action)
    
        obs, reward, done, info = env.step(env_action)
        
        for i, ff_episode in enumerate(ff_episodes):
            if len(ff_episode) > 0 and ff_episode.dones[-1] == True:
                continue
            dd = (reward[i] > 0)
            ff_episode.add_step(ff_action[i], obs[i], reward[i], dd, None)
        
        # full_obs = env_runner.env.call('get_attr', 'obs')
        for i, episode in enumerate(episodes):
            obss = info[i]['single_step_obs']
            rewards = info[i]['single_step_reward']
            l = len(obss)
            dones = [False] * (l-1) + [done[i]]
            for t in range(l):
                if len(episode) > 0 and episode.dones[-1] == True:
                    break
                dd = (rewards[t] > 0)
                episode.add_step(action[i, t], obss[t], rewards[t], dd, None)
        
        done = np.all(done | np.array([episode.dones[-1] for episode in episodes]))
        past_action = action
    
    # env.render()
    
    return episodes, ff_episodes


In [11]:
class AlwaysFFPolicy:
    def act(self, obs):
        return [1] * len(obs)
ff_policy = AlwaysFFPolicy()

In [ ]:
N = 10

all_episodes = []
success_rates = []
execution_times = []
for i in range(N):
    episodes, _ = collect_episodes_ff(policy, ff_policy, test_tasks)
    all_episodes.append(episodes)
    success_rate = np.mean([
        epi.rewards[-1] == 1
        for epi in episodes
    ])
    
    success_rates.append(success_rate)
    print(i, success_rate, np.mean(success_rates))
print('success rate', np.mean(success_rates), np.std(success_rates) / np.sqrt(N))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [05:13<00:00, 313.18s/it]


0 0.95 0.95


  0%|                                                                                                                  | 0/1 [00:00<?, ?it/s]